In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%%capture
!pip install datasets==1.4.1
!pip install transformers==4.4.0
!pip install torchaudio
!pip install librosa
!pip install jiwer

In [3]:
!pip install --upgrade pip
!pip install --upgrade datasets transformers accelerate soundfile librosa evaluate jiwer tensorboard gradio

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/15/aa/3f4c7bcee2057a76562a5b33ecbd199be08cdb4443a02e26bd2c3cf6fc39/pip-23.3.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 3.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.

In [4]:
!pip install librosa==0.9.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 3.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.1
    Uninstalling librosa-0.10.1:
      Successfully uninstalled librosa-0.10.1


In [5]:
import pandas as pd
english=pd.read_json("/kaggle/input/audios/HACKATHON_FILES/inen.json")

In [6]:
dict(english)

{'filepath': 0       ./HACKATHON/IN_ENGLISH/281474976895472_f74_chu...
 1       ./HACKATHON/IN_ENGLISH/281474976899770_f1412_c...
 2       ./HACKATHON/IN_ENGLISH/281474976903881_f1632_c...
 3       ./HACKATHON/IN_ENGLISH/281474976895472_f2996_c...
 4       ./HACKATHON/IN_ENGLISH/281474976899939_f968_ch...
                               ...                        
 6651    ./HACKATHON/IN_ENGLISH/281474976886701_f1840_c...
 6652    ./HACKATHON/IN_ENGLISH/281474976901602_f2119_c...
 6653    ./HACKATHON/IN_ENGLISH/281474976893606_f1423_c...
 6654    ./HACKATHON/IN_ENGLISH/281474976894336_f2182_c...
 6655    ./HACKATHON/IN_ENGLISH/281474976891494_f2262_c...
 Name: filepath, Length: 6656, dtype: object,
 'text': 0                                                      Up
 1                                                    Skip
 2                                                      Up
 3                                                      Up
 4                                               

In [13]:
train_dataset=english[:5500]
test_dataset=english[5501:]

In [14]:
train_dataset=train_dataset[train_dataset['duration']>1]
test_dataset=test_dataset[test_dataset['duration']>1]

In [12]:
train_dataset

,filepath,text,gender,duration,samplerate
1221,./HACKATHON/IN_ENGLISH/281474976893913_f2002_c...,and Nagaland,Male,1.000125,16000
1222,./HACKATHON/IN_ENGLISH/281474976891399_f2256_c...,between 10,Male,1.004125,16000
1223,./HACKATHON/IN_ENGLISH/281474976898607_f2144_c...,December 25,Male,1.004312,16000
1224,./HACKATHON/IN_ENGLISH/281474976893058_f2158_c...,The reply is,Male,1.006687,16000
1225,./HACKATHON/IN_ENGLISH/281474976894268_f1720_c...,financial health,Female,1.008000,16000
...,...,...,...,...,...
4995,./HACKATHON/IN_ENGLISH/281474976897527_f1838_c...,"It also develops a kind of team spirit, brothe...",Male,7.200437,16000
4996,./HACKATHON/IN_ENGLISH/281474976884109_f849_ch...,Can you stop automatically sending Samreen Rs....,Female,7.202813,16000
4997,./HACKATHON/IN_ENGLISH/281474976888205_f1855_c...,Most people here are engaged in businesses of ...,Female,7.204625,16000
4998,./HACKATHON/IN_ENGLISH/281474976893730_f1545_c...,Police have been told to stand down and are no...,Male,7.204625,16000


In [15]:
test_dataset

,filepath,text,gender,duration,samplerate
5501,./HACKATHON/IN_ENGLISH/281474976898387_f3126_c...,"Being hurt badly in hockey, I feel it is diffi...",Male,8.621187,16000
5502,./HACKATHON/IN_ENGLISH/281474976894945_f1678_c...,then the advice that was given to Santiago is ...,Female,8.626500,16000
5503,./HACKATHON/IN_ENGLISH/281474976900596_f3268_c...,The first global environmental forum from an I...,Male,8.626625,16000
5504,./HACKATHON/IN_ENGLISH/281474976892331_f1842_c...,The Vacation the most relaxing vacation that I...,Male,8.637500,16000
5505,./HACKATHON/IN_ENGLISH/281474976896698_f1923_c...,"But once I started practicing it, I generally ...",Female,8.638000,16000
...,...,...,...,...,...
6651,./HACKATHON/IN_ENGLISH/281474976886701_f1840_c...,"Actually, there's a book called a boy in the s...",Female,29.289437,16000
6652,./HACKATHON/IN_ENGLISH/281474976901602_f2119_c...,How do I decide composition of shot? That is p...,Female,29.440562,16000
6653,./HACKATHON/IN_ENGLISH/281474976893606_f1423_c...,In the academic year 2018 - 2019 list submitte...,Male,29.651812,16000
6654,./HACKATHON/IN_ENGLISH/281474976894336_f2182_c...,My district in my state has been improved econ...,Male,29.696500,16000


In [16]:
from datasets import Dataset, load_metric

df_train_dataset= Dataset.from_pandas(train_dataset)
print(df_train_dataset)
df_test_dataset= Dataset.from_pandas(test_dataset)
print(df_test_dataset)

Dataset({
    features: ['filepath', 'text', 'gender', 'duration', 'samplerate', '__index_level_0__'],
    num_rows: 4279
})
Dataset({
    features: ['filepath', 'text', 'gender', 'duration', 'samplerate'],
    num_rows: 1155
})


In [17]:
df_train_dataset=df_train_dataset.remove_columns(['gender'])
df_test_dataset=df_test_dataset.remove_columns(['gender'])

In [18]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = df_train_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=df_train_dataset.column_names)
vocab_test = df_test_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=df_test_dataset.column_names)

Map:   0%|          | 0/4279 [00:00<?, ? examples/s]

Map:   0%|          | 0/1155 [00:00<?, ? examples/s]

In [19]:
vocab_train

Dataset({
    features: ['vocab', 'all_text'],
    num_rows: 1
})

In [20]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
print(vocab_dict)

{'.': 0, 'e': 1, 'a': 2, '%': 3, '\n': 4, 'v': 5, 'P': 6, 'n': 7, 'E': 8, 'c': 9, 'I': 10, "'": 11, 'K': 12, 'S': 13, 'q': 14, 'H': 15, 'l': 16, 'z': 17, ':': 18, '?': 19, 'C': 20, '&': 21, 'o': 22, '\r': 23, 'x': 24, 'f': 25, '7': 26, '(': 27, '9': 28, '_': 29, '$': 30, '3': 31, '8': 32, 'O': 33, 'F': 34, ' ': 35, 'U': 36, 'b': 37, 'X': 38, '@': 39, 'V': 40, 'w': 41, 't': 42, '-': 43, 'p': 44, '1': 45, '5': 46, 'Y': 47, 'k': 48, '"': 49, '0': 50, 'M': 51, 'i': 52, ',': 53, '+': 54, '2': 55, 'D': 56, '!': 57, 'T': 58, 'r': 59, 'N': 60, 'u': 61, 'Z': 62, 'B': 63, 'L': 64, 'R': 65, 's': 66, 'A': 67, '6': 68, 'Q': 69, 'd': 70, 'g': 71, 'J': 72, 'G': 73, 'W': 74, 'y': 75, '4': 76, ')': 77, 'm': 78, 'h': 79, 'j': 80, ';': 81}


In [22]:
print(len(vocab_dict))

82


In [23]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [24]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

84


In [25]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [26]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


In [27]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [28]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [29]:
import torchaudio
import os
import librosa
import numpy as np
from pydub import AudioSegment

def apply_augmentation(audio, sampling_rate):
    # Example: Time Stretching/Compression
    speed_factor = np.random.uniform(0.7, 1.3)
    # Corrected: time_stretch takes only one argument
    audio = librosa.effects.time_stretch(y=audio, rate=speed_factor)

    # Example: Pitch Shifting
    pitch_factor = np.random.uniform(-2, 2)  # pitch shift up or down by 2 semitones
    audio = librosa.effects.pitch_shift(y=audio, sr=sampling_rate, n_steps=pitch_factor)

    # Example: Noise Injection
    noise = np.random.normal(0, 0.005, len(audio))  # adjust the standard deviation as needed
    audio = audio + noise

    # Example: Amplitude Scaling
    volume_factor = np.random.uniform(0.5, 1.5)
    audio = audio * volume_factor

    return audio

def speech_file_to_array_fn(batch):
    # Extract the directory path from the file path
    directory_path = "/kaggle/input/audios/HACKATHON_FILES/HACKATHON/IN_ENGLISH"

    # Extract the file name from the original path
    file_name = os.path.basename(batch["filepath"])

    # Remove the ".mp3" extension if present and add the ".wav" extension
#     file_name = os.path.splitext(file_name)[0] + ".wav"

    # Construct the full path by joining the directory path and the modified file name
    full_path = os.path.join(directory_path, file_name)
    
    try:
        # Attempt to load the audio file
        speech_array, sampling_rate = torchaudio.load(full_path)

        # Apply augmentation
        augmented_audio = apply_augmentation(speech_array[0].numpy(), sampling_rate)

        batch["speech"] = augmented_audio.tolist()  # Convert to list
        batch["sampling_rate"] = sampling_rate
        batch["target_text"] = batch["text"]
    except Exception as e:
        print(f"Error loading audio file {full_path}: {str(e)}")

    return batch

# Assuming train and test are your datasets
df_train_dataset = df_train_dataset.map(speech_file_to_array_fn, remove_columns=df_train_dataset.column_names)
df_test_dataset = df_test_dataset.map(speech_file_to_array_fn, remove_columns=df_test_dataset.column_names)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Map:   0%|          | 0/4279 [00:00<?, ? examples/s]

Map:   0%|          | 0/1155 [00:00<?, ? examples/s]

In [56]:
df_train_dataset[20]

{'speech': [-0.008097405551925374,
  -0.0005356800995163247,
  0.007356070487277749,
  -0.014548959096786327,
  0.0010444781236100154,
  -0.010568126779545925,
  -0.0056508124216316305,
  -0.008937711530008595,
  -0.00795876713813647,
  0.0064691659372430885,
  0.007322377524636236,
  0.0025498115788572537,
  0.0016466820790423714,
  -9.366627944643341e-05,
  -0.0029569841287999427,
  0.0036581044118010123,
  0.0025812522227447636,
  -0.0017444530310446265,
  0.006088883364253327,
  -0.0016737549276906035,
  0.003912360133823039,
  0.004435834958176054,
  -0.010147923771119814,
  -0.006251986150593783,
  -0.00039594552525405605,
  -0.004731258391118848,
  0.0009615097413746096,
  0.006935989879064743,
  -0.00856453642733389,
  -0.0011706234438882133,
  -0.006875732768144356,
  -0.00020253410113350796,
  0.0025749648632168816,
  -0.00984151477104892,
  0.0023339979420337257,
  -0.0025638776953062032,
  -0.0019932465301365555,
  0.001419607156690869,
  0.012733802259732986,
  0.002406969

In [30]:
import librosa
import numpy as np

def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), batch['sampling_rate'], 16_000)
    batch["sampling_rate"] = 16_000
    return batch

df_train_dataset = df_train_dataset.map(resample, num_proc=1)
df_test_dataset= df_test_dataset.map(resample, num_proc=1)

Map:   0%|          | 0/4279 [00:00<?, ? examples/s]

/tmp/ipykernel_42/412053579.py:5: FutureWarning: Pass orig_sr=16000, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  batch["speech"] = librosa.resample(np.asarray(batch["speech"]), batch['sampling_rate'], 16_000)


Map:   0%|          | 0/1155 [00:00<?, ? examples/s]

In [31]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(df_train_dataset))

ipd.Audio(data=np.asarray(df_train_dataset[rand_int]["speech"]), autoplay=True, rate=16_000)

In [34]:
rand_int = random.randint(0, len(df_train_dataset))

print("Target text:", df_train_dataset[rand_int]["target_text"])
print("Input array shape:", np.asarray(df_train_dataset[rand_int]["speech"]).shape)
print("Sampling rate:", df_train_dataset[rand_int]["sampling_rate"])

Target text: from where I always get inspired in the trace of my life.
Input array shape: (148863,)
Sampling rate: 16000


In [35]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [36]:
df_train_dataset = df_train_dataset.map(prepare_dataset, remove_columns=df_train_dataset.column_names, batch_size=8, num_proc=4, batched=True)
df_test_dataset = df_test_dataset.map(prepare_dataset, remove_columns=df_test_dataset.column_names, batch_size=8, num_proc=4, batched=True)

Map (num_proc=4):   0%|          | 0/4279 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

Map (num_proc=4):   0%|          | 0/1155 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

In [38]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [39]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True) 

In [79]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [41]:
from datasets import load_metric
wer_metric = load_metric("wer")

In [42]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    print(pred_str,label_str)
    return {"wer": wer}

In [43]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ignore_mismatched_sizes=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

config.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
model.freeze_feature_extractor()

/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1913: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [45]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/kaggle/working/",
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=10,
  fp16=True,
  save_steps=100, #this would mean every 400 steps model gets saved which also means Google drive gets full
  eval_steps=100,
  logging_steps=100,
  learning_rate=5e-5,
  # learning_rate=0.1, # this is just for demo
  warmup_steps=100,
  save_total_limit=2,
)

In [46]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=df_train_dataset,
    eval_dataset=df_test_dataset,
    tokenizer=processor.feature_extractor,
)

In [48]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
100,14.798000,6.694239,1.000000
200,4.557300,3.453694,1.000000
300,3.403900,3.391718,1.000000
400,3.451500,3.337294,1.000000
500,3.370500,3.307364,1.000000


...

/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.',

/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.',

/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
